# Create Azure Machine Learning Custom Environment

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../jobs/configuration.ipynb) 
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a custom environment from python SDK using
  - A docker image
  - A docker context
  - A docker image with a conda YAML

**Motivations** - Azure Machine Learning environments are an encapsulation of the environment where your machine learning training happens. By default your workspace has several curated environments already available. This notebook explains how to create a custom environment to run your specific task if you need to.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, BuildContext
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

OR

In [ ]:
ml_client = MLClient.from_config(DefaultAzureCredential())

# 2. Create Environment
Azure Machine Learning [environments](https://docs.microsoft.com/en-us/azure/machine-learning/concept-environments) are an encapsulation of the environment where your machine learning training happens. They specify the Python packages, environment variables, and software settings around your training and scoring scripts. They also specify run times (Python, Spark, or Docker). The environments are managed and versioned entities within your Machine Learning workspace that enable reproducible, auditable, and portable machine learning workflows across a variety of computes.

The workspace contains several curated environments by default to use as-is. However, you can create your own custom environment to meet your specific needs.

The `Environment` class will be used to create a custom environment. It accepts the following key parameters:
- `name` - Name of the environment.		
- `version`	- Version of the environment. If omitted, Azure ML will autogenerate a version.		
- `image` - The Docker image to use for the environment. Either `image` or `build` is required to create environment.
- `conda_file` - The standard conda YAML [configuration file](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-file-manually) of the dependencies for a conda environment. It can be used with a `image`. If specified, Azure ML will build the conda environment on top of the Docker image provided.
- `BuildContext`- The Docker build context configuration to use for the environment. Either `image` or `build` is required to create environment.
  - `path`- Local path to the directory to use as the build context.		
  - `dockerfile_path` - Relative path to the Dockerfile within the build context.
- `description`	- Description of the environment.		

## 2.1 Create a custom environment using a conda file
In this sample we will create an environment in Azure Machine Learning workspace using a docker image. The `Environment` is used to initialize the details. The `MLClient` is used to create the environment in the workspace.

#### Define the private wheel file URLs (for example, from Azure Blob Storage)

In [ ]:
private_wheel_1 = "https://<your_blob_storage_account>.blob.core.windows.net/<your_container>/your-private-wheel-1.whl"
private_wheel_2 = "https://<your_blob_storage_account>.blob.core.windows.net/<your_container>/your-private-wheel-2.whl"

In [ ]:
env = Environment(
    name="my-custom-env",
    image="mcr.microsoft.com/azureml/base:latest",  # You can specify a base image, such as the Azure ML base image
    conda_file={
        "dependencies": [
            "python=3.8",  # Specify the Python version
            "pip",
            {"pip": [private_wheel_1, private_wheel_2]},
        ]
    },
)

# Register the environment
ml_client.environments.create_or_update(env)

# 3. Define a job within the custom environment

In [ ]:
from azure.ai.ml.entities import Job
from azure.ai.ml import command

job = command(
    command="echo 'hello'",  # Your script or command
    code="./my-script-folder",  # Directory containing the script
    environment=env,  # The environment with private wheels
    compute="my-compute",  # Compute target to use
    environment_variables={"ENV_VAR": "value"},  # Optional: environment variables
)

# Submit the job
ml_client.jobs.create_or_update(job)